In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [2]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target

c:\Users\siddh\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# Fitness function for RandomForestClassifier
def fitness_rf_classifier(particle, X_train, X_test, y_train, y_test):
    rf_classifier = RandomForestClassifier(n_estimators=10, random_state=42)
    rf_classifier.fit(X_train.iloc[:, particle], y_train)

    y_pred = rf_classifier.predict(X_test.iloc[:, particle])

    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

In [5]:
def whale_optimization_algorithm(pop, X_train, X_test, y_train, y_test, pop_size, totalfeat, max_iterations):
    population = pop
    
    for iteration in tqdm(range(max_iterations)):
        for i in range(pop_size):
            a, A, C, l, p = np.random.uniform(-1, 1, 5)
            
            D = np.abs(C * population[int(p)].astype(int) - population[i].astype(int))
            X_rand = population[int(l)] - A * D
            new_solution = np.clip(X_rand, 0, totalfeat - 1).astype(int)
            
            fitness_current = fitness_rf_classifier(population[i], X_train, X_test, y_train, y_test)
            fitness_new = fitness_rf_classifier(new_solution, X_train, X_test, y_train, y_test)
            
            if fitness_new > fitness_current:
                population[i] = new_solution
    
    best_individual = population[np.argmax([fitness_rf_classifier(ind, X_train, X_test, y_train, y_test) for ind in population])]
    return best_individual

In [7]:
# Example usage:
# Assuming X_train, y_train, X_test, and y_test are already defined
num_features = 28  # Change this according to your actual number of features
pop_size = 16
max_iter = 10

num_particles = 16
col_len = 28
max_value = 783
std_dev = 150
mean = max_value / 2
pop_size = (num_particles, col_len)

population = np.random.normal(mean, std_dev, pop_size).astype(int)
population[population > max_value] = max_value
population[population < 0] = 0

totalfeat = 784

In [11]:
population

array([722, 469, 277, 565, 357, 309, 267, 355, 384, 295, 433, 268, 374,
       245, 583, 465,  47, 345, 337,  68, 403, 623, 206, 379, 369, 617,
       128, 368])

In [12]:
# # Initialize population
# population = np.random.randint(0, totalfeat, size=(pop_size, num_features))

# Run the WOA algorithm
best_solution = whale_optimization_algorithm(population, X_train, X_test, y_train, y_test, pop_size[0], totalfeat, max_iter)

print("\nBest solution found:")
print(best_solution)
accuracy = fitness_rf_classifier(best_solution, X_train, X_test, y_train, y_test)
print("Accuracy of best solution = %.6f" % accuracy)

100%|██████████| 10/10 [09:41<00:00, 58.15s/it]



Best solution found:
[132 240 321 301 327 408 291 220 222 432 238 284 573 147 289 409 405 381
 483 514 307 281 601 516 459 368 271 491]
Accuracy of best solution = 0.845643
